In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sys

import random
import math
import dalmatian
import matplotlib.pyplot as plt
from statsmodels.stats import multitest
import scipy.stats as stat

In [2]:
subclones_sig_genes = pd.read_csv('~/Documents/2025/CloneSig_Test/sig_genes_subclones.txt',sep='\t')
subset=subclones_sig_genes[subclones_sig_genes['p'] <= 0.001]
subset_genes_list = subset['gene']

In [3]:
subset

,gene,longname,codelen,nnei,nncd,nsil,nmis,nstp,nspl,nind,...,npat,nsite,pCV,pCL,pFN,pCL2,pFN2,pCF,p,q
0,ESR1,estrogen receptor 1,1901,48,1,2,8,0,0,1,...,9,5,1.218289e-10,0.000001,0.00166,2.304368e-05,0.001680,NaN,4.551914e-15,8.276291e-11
1,KMT2C,myeloid/lymphoid or mixed-lineage leukemia 3,15143,51,1,0,9,11,0,1,...,16,21,1.985128e-11,1.000000,0.74100,1.000000e+00,0.697105,NaN,5.090414e-10,4.627695e-06
2,TP53,tumor protein p53,1422,42,0,0,3,1,1,0,...,5,5,1.142566e-06,1.000000,0.07800,1.000000e+00,0.073564,NaN,1.867480e-06,1.131817e-02
3,KRAS,v-Ki-ras2 Kirsten rat sarcoma viral oncogene h...,727,174,0,0,3,0,0,0,...,3,2,4.243216e-05,0.005200,0.25680,4.094628e-03,0.255962,NaN,3.602442e-06,1.637490e-02
4,PIK3CA,"phosphoinositide-3-kinase, catalytic, alpha po...",3283,71,0,0,6,0,0,0,...,5,6,2.318574e-05,1.000000,0.03500,1.000000e+00,0.037330,NaN,2.664059e-05,9.687583e-02
5,MNDA,myeloid cell nuclear differentiation antigen,1248,240,1,0,3,0,0,0,...,3,3,1.537381e-04,1.000000,0.00850,1.000000e+00,0.007809,NaN,4.112391e-05,1.246192e-01
6,CTNNA2,"catenin (cadherin-associated protein), alpha 2",3166,23,1,1,4,0,0,0,...,4,2,3.832101e-02,0.000100,0.97867,3.969570e-07,0.978592,NaN,5.162644e-05,1.340960e-01
7,HIST1H3B,"histone cluster 1, H3b",413,224,0,0,2,0,0,0,...,2,1,3.643148e-04,0.067000,0.47000,2.409639e-03,0.472656,NaN,2.836472e-04,6.446591e-01
8,CSAD,cysteine sulfinic acid decarboxylase,1629,95,1,0,0,2,0,0,...,2,2,1.089837e-03,1.000000,0.02100,9.999996e-01,0.026402,NaN,4.703418e-04,9.501950e-01
9,ST18,suppression of tumorigenicity 18 (breast carci...,3221,72,2,0,3,0,0,0,...,3,2,4.784297e-02,0.001678,0.00173,9.303287e-04,0.001715,NaN,5.237650e-04,9.508162e-01


## on clones 100k Mutsig genes

In [21]:
one_sided_clones_all_genes = pd.read_csv('~/Documents/2025/CloneSig_Test/results/Cb_onesided_clones_040325_100k.txt',sep='\t')
one_sided_clones_all_genes.drop(columns=['Cb_exact_p_vals'],inplace=True)
one_sided_clones_all_genes = one_sided_clones_all_genes.sort_values(by=['Cb_p_vals'])

In [22]:
number_pats_clones_dist = pd.read_csv("~/Documents/2025/CloneSig_Test/pats_clones_mutation_table_2k_genes_clones_040425.tsv",sep='\t')
number_pats_clones_dist.rename(columns={'Hugo_Symbol':'gene'},inplace=True)

In [23]:
one_sided_clones_all_genes

,gene,Cb_p_vals
1,ESR1,0.00010
5,MEN1,0.00102
2,KMT2C,0.00224
9,CTCF,0.05639
7,HIST1H3B,0.17531
11,CSAD,0.18488
0,PIK3CA,0.42612
8,ST18,0.48366
10,ZNF536,0.51295
3,TP53,0.76899


In [24]:
Cb_with_counts = one_sided_clones_all_genes.merge(number_pats_clones_dist, how='left',on='gene')
Cb_with_counts

,gene,Cb_p_vals,n_mut_pats,n_mut_clones,n_total_muts
0,ESR1,0.00010,6,10,10
1,MEN1,0.00102,1,2,2
2,KMT2C,0.00224,5,18,21
3,CTCF,0.05639,2,4,5
4,HIST1H3B,0.17531,2,3,3
5,CSAD,0.18488,2,3,3
6,PIK3CA,0.42612,11,15,19
7,ST18,0.48366,4,5,5
8,ZNF536,0.51295,4,5,5
9,TP53,0.76899,5,7,7


In [26]:
results_mutsig_one_sided_clones_all_genes = Cb_with_counts[Cb_with_counts['gene'].isin(subset_genes_list)]
_, results_mutsig_one_sided_clones_all_genes["CloneSig_q"], _, _ = multitest.multipletests(results_mutsig_one_sided_clones_all_genes["Cb_p_vals"], method = "fdr_bh")
results_mutsig_one_sided_clones_all_genes.sort_values(by=['CloneSig_q'])

,gene,Cb_p_vals,n_mut_pats,n_mut_clones,n_total_muts,CloneSig_q
0,ESR1,0.00010,6,10,10,0.001200
1,MEN1,0.00102,1,2,2,0.006120
2,KMT2C,0.00224,5,18,21,0.008960
3,CTCF,0.05639,2,4,5,0.169170
4,HIST1H3B,0.17531,2,3,3,0.369760
5,CSAD,0.18488,2,3,3,0.369760
6,PIK3CA,0.42612,11,15,19,0.683933
7,ST18,0.48366,4,5,5,0.683933
8,ZNF536,0.51295,4,5,5,0.683933
9,TP53,0.76899,5,7,7,0.922788


In [9]:
genes_in_both = set(results_mutsig_one_sided_clones_all_genes['gene']) & set(subset_genes_list)
mutsig_to_add = subset[subset['gene'].isin(genes_in_both)]
mutsig_to_add = mutsig_to_add[['gene','p','q']]
mutsig_to_add.rename(columns={'p':'MutSig_p','q':'Mutsig_q'},inplace=True)

In [10]:
## fisher sum
combined_clones_df=results_mutsig_one_sided_clones_all_genes.merge(mutsig_to_add,on='gene')
combined_pvals=[]
for i,row in combined_clones_df.iterrows():
    p_val_list= [ row['MutSig_p'],row['Cb_p_vals'] ]
    test_stat, combined_p =stat.combine_pvalues(p_val_list, method='fisher',weights=None)
    combined_pvals.append(combined_p)
    
combined_clones_df['Fisher_Combined_p'] = combined_pvals

/opt/anaconda3/envs/py39/lib/python3.9/site-packages/scipy/stats/_stats_py.py:9638: RuntimeWarning: divide by zero encountered in log
  statistic = -2 * np.sum(np.log(pvalues))


In [11]:
combined_clones_df.sort_values('Fisher_Combined_p',ascending=True)

,gene,Cb_p_vals,CloneSig_q,MutSig_p,Mutsig_q,Fisher_Combined_p
0,ESR1,0.0000,0.000000,4.551914e-15,8.276291e-11,0.000000e+00
2,KMT2C,0.0029,0.011600,5.090414e-10,4.627695e-06,4.169072e-11
1,MEN1,0.0011,0.006600,6.668605e-04,9.508162e-01,1.109517e-05
9,TP53,0.5856,0.702720,1.867480e-06,1.131817e-02,1.610434e-05
11,KRAS,1.0000,1.000000,3.602442e-06,1.637490e-02,4.875508e-05
6,PIK3CA,0.4171,0.685067,2.664059e-05,9.687583e-02,1.378696e-04
3,CTCF,0.0582,0.174600,6.798268e-04,9.508162e-01,4.406671e-04
10,MNDA,1.0000,1.000000,4.112391e-05,1.246192e-01,4.564310e-04
4,HIST1H3B,0.1774,0.380400,2.836472e-04,6.446591e-01,5.483327e-04
5,CSAD,0.1902,0.380400,4.703418e-04,9.501950e-01,9.233718e-04


In [26]:
_, combined_clones_df["Fisher_Combined_q"], _, _ = multitest.multipletests(combined_clones_df["Fisher_Combined_p"], method = "fdr_bh")
combined_clones_df.sort_values(by=['Fisher_Combined_q'])


,gene,Cb_p_vals,CloneSig_q,MutSig_p,Mutsig_q,Fisher_Combined_p,Fisher_Combined_q
0,ESR1,0.0000,0.000000,4.551914e-15,8.276291e-11,0.000000e+00,0.000000e+00
2,KMT2C,0.0029,0.011600,5.090414e-10,4.627695e-06,4.169072e-11,2.501443e-10
1,MEN1,0.0011,0.006600,6.668605e-04,9.508162e-01,1.109517e-05,4.438067e-05
9,TP53,0.5856,0.702720,1.867480e-06,1.131817e-02,1.610434e-05,4.831302e-05
11,KRAS,1.0000,1.000000,3.602442e-06,1.637490e-02,4.875508e-05,1.170122e-04
6,PIK3CA,0.4171,0.685067,2.664059e-05,9.687583e-02,1.378696e-04,2.757392e-04
3,CTCF,0.0582,0.174600,6.798268e-04,9.508162e-01,4.406671e-04,6.846465e-04
10,MNDA,1.0000,1.000000,4.112391e-05,1.246192e-01,4.564310e-04,6.846465e-04
4,HIST1H3B,0.1774,0.380400,2.836472e-04,6.446591e-01,5.483327e-04,7.311103e-04
5,CSAD,0.1902,0.380400,4.703418e-04,9.501950e-01,9.233718e-04,1.108046e-03


## on subclones mutsig genes results

In [38]:
one_sided_subclones_all_genes = pd.read_csv('~/Documents/2025/CloneSig_Test/results/Cb_onesided_subclones_040625_100k.txt',sep='\t')
one_sided_subclones_all_genes.drop(columns=['Cb_exact_p_vals'],inplace=True)
one_sided_subclones_all_genes = one_sided_subclones_all_genes.sort_values(by=['Cb_p_vals'])

In [39]:
number_pats_subclones_dist = pd.read_csv("~/Documents/2025/CloneSig_Test/pats_clones_mutation_table_2k_genes_subclones_040425.tsv",sep='\t')
number_pats_subclones_dist.rename(columns={'Hugo_Symbol':'gene'},inplace=True)

In [40]:
Cb_subclones_with_counts = one_sided_subclones_all_genes.merge(number_pats_subclones_dist, how='left',on='gene')
Cb_subclones_with_counts

,gene,Cb_p_vals,n_mut_pats,n_mut_clones,n_total_muts
0,ESR1,0.00001,5,9,9
1,MEN1,0.00089,1,2,2
2,KMT2C,0.00860,5,17,20
3,CTCF,0.06138,2,4,5
4,CSAD,0.19089,2,3,3
5,ST18,0.28319,3,4,4
6,ZNF536,0.53785,4,5,5
7,TP53,0.90461,5,6,6
8,PIK3CA,1.00000,4,4,5
9,KRAS,1.00000,3,3,3


In [41]:
results_mutsig_one_sided_subclones_all_genes = Cb_subclones_with_counts[Cb_subclones_with_counts['gene'].isin(subset_genes_list)]
_, results_mutsig_one_sided_subclones_all_genes["CloneSig_q"], _, _ = multitest.multipletests(results_mutsig_one_sided_subclones_all_genes["Cb_p_vals"], method = "fdr_bh")
results_mutsig_one_sided_subclones_all_genes.sort_values(by=['CloneSig_q'])

,gene,Cb_p_vals,n_mut_pats,n_mut_clones,n_total_muts,CloneSig_q
0,ESR1,0.00001,5,9,9,0.000120
1,MEN1,0.00089,1,2,2,0.005340
2,KMT2C,0.00860,5,17,20,0.034400
3,CTCF,0.06138,2,4,5,0.184140
4,CSAD,0.19089,2,3,3,0.458136
5,ST18,0.28319,3,4,4,0.566380
6,ZNF536,0.53785,4,5,5,0.922029
7,TP53,0.90461,5,6,6,1.000000
8,PIK3CA,1.00000,4,4,5,1.000000
9,KRAS,1.00000,3,3,3,1.000000


In [42]:
genes_in_both = set(results_mutsig_one_sided_subclones_all_genes['gene']) & set(subset_genes_list)
mutsig_to_add = subset[subset['gene'].isin(genes_in_both)]
mutsig_to_add = mutsig_to_add[['gene','p','q']]
mutsig_to_add.rename(columns={'p':'MutSig_p','q':'Mutsig_q'},inplace=True)

In [32]:
## fisher sum
combined_subclones_df=results_mutsig_one_sided_subclones_all_genes.merge(mutsig_to_add,on='gene')
combined_pvals=[]
for i,row in combined_subclones_df.iterrows():
    p_val_list= [ row['MutSig_p'],row['Cb_p_vals'] ]
    test_stat, combined_p =stat.combine_pvalues(p_val_list, method='fisher',weights=None)
    combined_pvals.append(combined_p)
    
combined_subclones_df['Fisher_Combined_p'] = combined_pvals

/opt/anaconda3/envs/py39/lib/python3.9/site-packages/scipy/stats/_stats_py.py:9638: RuntimeWarning: divide by zero encountered in log
  statistic = -2 * np.sum(np.log(pvalues))


In [33]:
combined_subclones_df.sort_values('Fisher_Combined_p',ascending=True)

,gene,Cb_p_vals,CloneSig_q,MutSig_p,Mutsig_q,Fisher_Combined_p
0,ESR1,0.0000,0.000000,4.551914e-15,8.276291e-11,0.000000e+00
2,KMT2C,0.0081,0.032400,5.090414e-10,4.627695e-06,1.122113e-10
1,MEN1,0.0006,0.003600,6.668605e-04,9.508162e-01,6.294434e-06
7,TP53,0.7754,1.000000,1.867480e-06,1.131817e-02,2.091743e-05
11,KRAS,1.0000,1.000000,3.602442e-06,1.637490e-02,4.875508e-05
9,PIK3CA,1.0000,1.000000,2.664059e-05,9.687583e-02,3.072479e-04
3,CTCF,0.0593,0.177900,6.798268e-04,9.508162e-01,4.482410e-04
10,MNDA,1.0000,1.000000,4.112391e-05,1.246192e-01,4.564310e-04
4,CSAD,0.1869,0.448560,4.703418e-04,9.501950e-01,9.088897e-04
5,ST18,0.2939,0.587800,5.237650e-04,9.508162e-01,1.505323e-03


In [43]:
with pd.ExcelWriter('Extended_Data_Table_CloneSig_Res_040625.xlsx') as writer:  
    results_mutsig_one_sided_clones_all_genes.to_excel(writer, sheet_name='CloneSig_With_Trunk')
    results_mutsig_one_sided_subclones_all_genes.to_excel(writer, sheet_name='CloneSig_Subclones')

    
    